- Image Folder
- Scheduler
- Transfer Learning

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [43]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [44]:
data_transforms = {
    "train" : transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    "val" : transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

Download the dataset from <a href="https://www.youtube.com/redirect?event=video_description&redir_token=QUFFLUhqbVhUbHhxa05pWm5IeVctdkVOU2dfWVRtSHltQXxBQ3Jtc0tsVUZ6VFB6eGF0UE1yTmNEN2I0M3JxWTdjNWhHc3dPbEs4WE1TRVQ0aE5nU19DNzc0Z0MyeS16blkwQUl1STBRdU1xWkJ4YzhXTWM1TXIxT01YMVRVRExGTlRnTDBhVHUyc0pUVkpjUFlCQzNJcGRTNA&q=https%3A%2F%2Fdownload.pytorch.org%2Ftutorial%2Fhymenoptera_data.zip&v=K0lWSB2QoIQ">here</a> and extract in data folder

In [45]:
data_dir = "data/hymenoptera_data"
sets = ["train", "val"]
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x),
                            data_transforms[x])
        for x in ["train", "val"]
}

In [46]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size=4,
                                              shuffle=True,
                                              num_workers=0)
                for x in ['train', 'val']
               }

In [47]:
dataset_sizes = {x: len(image_datasets[x])
                 for x in ['train', 'val']
                 }

In [48]:
class_names = image_datasets['train'].classes
print(class_names)

['ants', 'bees']


In [49]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs-1}") 
        print("-" * 10)

        # each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()   # set model to training mode
            else:
                model.eval()    # set model to evaluation mode
            
            running_loss = 0.0
            running_corrects = 0.0

            # Iterate over data 
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # training history if only in train
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    #backward + optimize only if in training loop
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                    
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == "train":
                scheduler.step()
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s")
    print(f"Best val Acc: {best_acc:4f}")

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [50]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 2)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [51]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(
    params=model.parameters(),
    lr=0.001
)

In [54]:
# schedular
step_lr_schedular = lr_scheduler.StepLR(
    optimizer=optimizer,
    step_size=7,
    gamma=0.1
)
# after every 7 epoch, the learning rate will be multiplied with gamma

In [53]:
# for epoch in range(100):
    # train() # optimizer.step()
    # evaluate()
    # scheduler.step()
model = train_model(model, criterion, optimizer, step_lr_schedular, num_epochs=20)

Epoch 0/19
----------
train Loss: 0.6039 Acc: 0.6680
val Loss: 0.4488 Acc: 0.8301

Epoch 1/19
----------
train Loss: 0.4947 Acc: 0.7664
val Loss: 0.3417 Acc: 0.8954

Epoch 2/19
----------
train Loss: 0.5396 Acc: 0.7172
val Loss: 0.3100 Acc: 0.9150

Epoch 3/19
----------
train Loss: 0.4262 Acc: 0.7992
val Loss: 0.2567 Acc: 0.9412

Epoch 4/19
----------
train Loss: 0.4039 Acc: 0.8238
val Loss: 0.2388 Acc: 0.9281

Epoch 5/19
----------
train Loss: 0.4004 Acc: 0.8156
val Loss: 0.2214 Acc: 0.9412

Epoch 6/19
----------
train Loss: 0.3903 Acc: 0.8361
val Loss: 0.2107 Acc: 0.9412

Epoch 7/19
----------
train Loss: 0.4299 Acc: 0.8074
val Loss: 0.2087 Acc: 0.9346

Epoch 8/19
----------
train Loss: 0.3763 Acc: 0.8279
val Loss: 0.2093 Acc: 0.9412

Epoch 9/19
----------
train Loss: 0.4019 Acc: 0.8115
val Loss: 0.2030 Acc: 0.9542

Epoch 10/19
----------
train Loss: 0.3393 Acc: 0.8607
val Loss: 0.2238 Acc: 0.9477

Epoch 11/19
----------
train Loss: 0.3465 Acc: 0.8607
val Loss: 0.2181 Acc: 0.9281

Ep